In [ ]:
import numpy as np
from subprocess import PIPE, run
import matplotlib.pyplot as plt
import os
import textwrap

class ExptBuilder():
    def __init__(self):
        self.__code_path__ = os.environ.get('code')
        self.__temp_exp_path__ = os.path.join(self.__code_path__, "k-exp", "kexp", "experiments", "ml_expt.py")

    def run_expt(self):
        expt_path = self.__temp_exp_path__
        run_expt_command = r"%kpy% & artiq_run " + expt_path
        result = run(run_expt_command, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
        print(result.returncode, result.stdout, result.stderr)
        os.remove(self.__temp_exp_path__)
        return result.returncode
    
    def write_experiment_to_file(self, program):
        with open(self.__temp_exp_path__, 'w') as file:
            file.write(program)
    
    def fringe_scan_expt(self, t_amp_ramp):
        script = textwrap.dedent(f"""
        from artiq.experiment import *
        from artiq.experiment import delay
        from kexp import Base
        import numpy as np
        from kexp.calibrations import high_field_imaging_detuning
        from kexp import Base, img_types, cameras
        from kexp.calibrations.tweezer import tweezer_vpd1_to_vpd2

        from artiq.coredevice.shuttler import DCBias, DDS, Relay, Trigger, Config, shuttler_volt_to_mu

        T32 = 1<<32

        class mag_trap(EnvExperiment, Base):

            def prepare(self):
                Base.__init__(self,setup_camera=True,save_data=True,
                            camera_select=cameras.andor,
                            imaging_type=img_types.ABSORPTION)
                
                self.p.t_tof = 1500.e-6
                # self.xvar('t_tof',np.linspace(30.,800.,10)*1.e-6)

                # self.xvar('i_hf_tweezer_load_current',np.linspace(188.,193.,15))
                self.p.i_hf_tweezer_load_current = 190.5
        
                self.p.v_pd_hf_tweezer_1064_ramp_end = 9.
                self.p.v_tweezer_paint_amp_max = 3.14

                self.p.i_hf_tweezer_evap1_current = 194.3
                self.p.v_pd_hf_tweezer_1064_rampdown_end = 1.
                self.p.t_hf_tweezer_1064_rampdown = 100.e-3        

                self.p.i_hf_tweezer_evap2_current = 193.667
                self.p.v_pd_hf_tweezer_1064_rampdown2_end = .12
                self.p.t_hf_tweezer_1064_rampdown2 = 700.e-3  

                self.p.i_hf_tweezer_evap3_current = 193.7 
                self.p.v_pd_hf_tweezer_1064_rampdown3_end = 1.
                self.p.t_hf_tweezer_1064_rampdown3 = 530.e-3   

                self.p.frequency_tweezer_list = [73.6e6,76.e6]

                a_list = [.16,.17]
                self.p.amp_tweezer_list = a_list

                self.p.amp_imaging = .1
                self.p.imaging_state = 2.

                self.p.N_repeats = 1
                self.p.t_mot_load = 1.

                self.finish_prepare(shuffle=True)

            @kernel
            def scan_kernel(self):

                self.set_high_field_imaging(i_outer=self.p.i_hf_tweezer_evap3_current)
                self.dds.imaging.set_dds(amplitude=self.p.amp_imaging)

                # self.switch_d2_2d(1)
                self.mot(self.p.t_mot_load)
                self.dds.push.off()
                self.cmot_d1(self.p.t_d1cmot * s)
                
                self.gm(self.p.t_gm * s)
                self.gm_ramp(self.p.t_gmramp)

                self.magtrap_and_load_lightsheet(do_magtrap_rampup=False)

                self.outer_coil.on()
                self.outer_coil.set_voltage()
                self.outer_coil.ramp_supply(t=self.p.t_feshbach_field_rampup,
                                    i_start=0.,
                                    i_end=self.p.i_hf_lightsheet_evap1_current)
                
                self.set_shims(v_zshim_current=0.,
                                v_yshim_current=0.,
                                v_xshim_current=0.)
                
                # lightsheet evap 1
                self.lightsheet.ramp(t=self.p.t_hf_lightsheet_rampdown,
                                    v_start=self.p.v_pd_lightsheet_rampup_end,
                                    v_end=self.p.v_pd_hf_lightsheet_rampdown_end)
                
                # self.outer_coil.ramp_supply(t=self.p.t_feshbach_field_ramp,
                #                      i_start=self.p.i_hf_lightsheet_evap1_current,
                #                      i_end=self.p.i_hf_tweezer_load_current)

                self.tweezer.on()
                self.tweezer.ramp(t=self.p.t_hf_tweezer_1064_ramp,
                                v_start=0.,
                                v_end=self.p.v_pd_hf_tweezer_1064_ramp_end,
                                paint=True,keep_trap_frequency_constant=False)
                                
                # lightsheet ramp down (to off)
                self.lightsheet.ramp(t=self.p.t_hf_lightsheet_rampdown2,
                                        v_start=self.p.v_pd_hf_lightsheet_rampdown_end,
                                        v_end=self.p.v_pd_lightsheet_rampdown2_end)

                self.lightsheet.off()
            
                # delay(self.p.t_lightsheet_hold)

                self.outer_coil.ramp_supply(t=self.p.t_feshbach_field_ramp,
                                    i_start=self.p.i_hf_lightsheet_evap1_current,
                                    i_end=self.p.i_hf_tweezer_evap1_current)

                # tweezer evap 1 with constant trap frequency
                self.tweezer.ramp(t=self.p.t_hf_tweezer_1064_rampdown,
                                v_start=self.p.v_pd_hf_tweezer_1064_ramp_end,
                                v_end=self.p.v_pd_hf_tweezer_1064_rampdown_end,
                                paint=True,keep_trap_frequency_constant=True)
                
                self.outer_coil.ramp_supply(t=self.p.t_feshbach_field_ramp,
                                    i_start=self.p.i_hf_tweezer_evap1_current,
                                    i_end=self.p.i_hf_tweezer_evap2_current)
                
                self.tweezer.ramp(t=self.p.t_hf_tweezer_1064_rampdown2,
                                v_start=self.p.v_pd_hf_tweezer_1064_rampdown_end,
                                v_end=self.p.v_pd_hf_tweezer_1064_rampdown2_end,
                                paint=True,keep_trap_frequency_constant=True)
                
                self.outer_coil.ramp_supply(t=self.p.t_feshbach_field_ramp,
                                    i_start=self.p.i_hf_tweezer_evap2_current,
                                    i_end=self.p.i_hf_tweezer_evap3_current)
                
                delay(2.e-3)
                self.ttl.pd_scope_trig.pulse(1.e-6)
                # tweezer evap 3 with constant trap frequency
                self.tweezer.ramp(t=self.p.t_hf_tweezer_1064_rampdown3,
                                v_start=tweezer_vpd1_to_vpd2(self.p.v_pd_hf_tweezer_1064_rampdown2_end),
                                v_end=self.p.v_pd_hf_tweezer_1064_rampdown3_end,
                                paint=True,keep_trap_frequency_constant=True,low_power=True)
                
                delay(self.p.t_tweezer_hold)
                
                self.tweezer.off()

                delay(self.p.t_tof)
                
                self.abs_image()

                self.outer_coil.off()

            @kernel
            def run(self):
                self.init_kernel()
                self.load_2D_mot(self.p.t_2D_mot_load_delay)
                self.scan()
                # self.mot_observe()

            def analyze(self):
                import os
                expt_filepath = os.path.abspath(__file__)
                self.end(expt_filepath)

                """)
        return script

In [2]:
eBuilder = ExptBuilder()

In [3]:
ts = np.linspace(20.e-3,2000.e-3,10)
# np.random.shuffle(ts)
for t in ts:
    print(t)
    eBuilder.write_experiment_to_file(eBuilder.fringe_scan_expt(t_amp_ramp=t))
    eBuilder.run_expt()

0.006153846153846154
0  200 values of t_tunnel. 200 total shots. 600 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 19634
Parameters saved, data closed.
Done!
 
0.026666666666666665
0  200 values of t_tunnel. 200 total shots. 600 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 19635
Parameters saved, data closed.
Done!
 
0.03692307692307692
0  200 values of t_tunnel. 200 total shots. 600 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 19636
Parameters saved, data closed.
Done!
 
0.07179487179487179
0  200 values of t_tunnel. 200 total shots. 600 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 19637
Parameters saved, data closed.
Done!
 
0.03282051282051282
0  200 values of t_tunnel. 200 total shots. 600 total images expected.
andor
Acknowledged camera ready signal.
Camera is ready.
 Run ID: 19638
Parameters saved, data clos

In [4]:
from kexp.control.relay_control import EthernetRelay
relay = EthernetRelay()

In [5]:
relay.source_off()